In [65]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [48]:
from langchain_community.document_loaders import PyPDFLoader

document_path = [r'data\jurisprudencia\STJ_202201522622_tipo_integra_178204788.pdf',
                 r'data\lei\lei-13709-14-agosto-2018-787077-normaatualizada-pl.pdf']
docs = []
for file in document_path:
    loader = PyPDFLoader(file)
    doc = loader.load()
    docs.extend(doc)
print(f'Número de documentos carregados: {len(docs)}')
print(docs[0].metadata)


Ignoring wrong pointing object 1 65536 (offset 0)
Ignoring wrong pointing object 26 65536 (offset 0)
Ignoring wrong pointing object 31 65536 (offset 0)
Ignoring wrong pointing object 54 65536 (offset 0)
Ignoring wrong pointing object 57 65536 (offset 0)
Ignoring wrong pointing object 60 65536 (offset 0)
Ignoring wrong pointing object 63 65536 (offset 0)
Ignoring wrong pointing object 66 65536 (offset 0)
Ignoring wrong pointing object 69 65536 (offset 0)
Ignoring wrong pointing object 72 65536 (offset 0)
Ignoring wrong pointing object 75 65536 (offset 0)
Ignoring wrong pointing object 79 65536 (offset 0)


Número de documentos carregados: 40
{'producer': 'iText 2.1.5 (by lowagie.com)', 'creator': 'PyPDF', 'creationdate': '2025-05-12T10:58:36-03:00', 'moddate': '2025-05-12T10:58:36-03:00', 'source': 'data\\jurisprudencia\\STJ_202201522622_tipo_integra_178204788.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1'}


In [49]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

## Testar:
## chunk_size = 500, 1000, 1500, 2000
## chunk_overlap = 100, 150, 200, 300
split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    separators=["\n\n\n", "\n\n", "\n", ".", ";", ","]
)
splitted_docs = split.split_documents(docs)
print(f"Total de chunks {len(splitted_docs)}")

Total de chunks 253


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
import os

os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

#HuggingFace : rufimelo/Legal-BERTimbau-large ou rufimelo/Legal-BERTimbau-base
#embeddings = HuggingFaceEmbeddings(model_name="neuralmind/bert-base-portuguese-cased")

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

In [51]:
import faiss
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(splitted_docs, 
                                    embedding=embeddings,
                                    )

vector_store.save_local(folder_path="data",
                        index_name="index_db")


## Retriever

In [52]:
## Pergunta do usuário
question = """
QUal é a jurisprudência atual sobre a LGPD ?
"""

In [53]:
## LLM para o retriever
from langchain_openai import ChatOpenAI
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-4o")

In [54]:
from typing import List

from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field


# Output parser will split the LLM result into a list of queries
class LineListOutputParser(BaseOutputParser[List[str]]):
    """Output parser for a list of lines."""

    def parse(self, text: str) -> List[str]:
        lines = text.strip().split("\n")
        return list(filter(None, lines))  # Remove empty lines


output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""Você é um assistente jurídico especializado em LGPD. Sua tarefa é gerar cinco 
    versões diferentes da pergunta do usuário para recuperar documentos relevantes sobre LGPD 
    de uma base de dados vetorial. Gere perspectivas alternativas focadas em:
    1. Aspectos legais específicos
    2. Jurisprudência relacionada
    3. Direitos dos titulares
    4. Obrigações dos controladores
    5. Penalidades e sanções
    
    Forneça essas perguntas alternativas separadas por quebras de linha.
    Pergunta original: {question}""",
)

# Chain
llm_chain = QUERY_PROMPT | llm | output_parser

In [55]:
from langchain.retrievers.multi_query import MultiQueryRetriever

# Run
retriever = MultiQueryRetriever(
    retriever=vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5}  # Aumentado para mais contexto
    ), 
    llm_chain=llm_chain, 
    parser_key="lines"
)

# Results
unique_docs = retriever.invoke(question)
len(unique_docs)

20

In [56]:
from langchain_openai import ChatOpenAI
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-4o")

In [61]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

system_prompt = """
    Você é um assistente jurídico especializado na Lei Geral de Proteção de Dados (LGPD - Lei nº 13.709/2018).

INSTRUÇÕES PARA RESPOSTA:
1. Use as informações disponíveis**: Trabalhe com as informações presentes nos documentos, mesmo que sejam parciais
2. Seja útil: Forneça o máximo de informação relevante disponível nos documentos
3. Contextualize: Explique conceitos jurídicos quando necessário
4. Cite fontes: Quando possível, mencione artigos da LGPD ou referências jurisprudenciais
5. Seja claro sobre limitações: Se algum aspecto específico não estiver detalhado, mencione isso, mas forneça o que está disponível

IMPORTANTE: Os documentos podem conter informações fragmentadas. Use seu conhecimento jurídico para interpretar e conectar as informações de forma coerente.

Apenas declare que a informação não está disponível se realmente NÃO houver NENHUMA informação relevante nos documentos fornecidos.

CONTEXTO DOS DOCUMENTOS:
{context}
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}")
])

In [62]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

# Criar chains
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

print("✓ Sistema RAG configurado com sucesso!")


✓ Sistema RAG configurado com sucesso!


In [63]:
try:
    result = retrieval_chain.invoke({
        "input": question,
        "chat_history": []  # Histórico vazio para primeira consulta
    })
    
    print("=== PERGUNTA ===")
    print(question)
    
    print("\n=== RESPOSTA ===")
    print(result["answer"])
    
    print(f"\n=== DOCUMENTOS UTILIZADOS ({len(result['context'])}) ===")
    for i, doc in enumerate(result["context"][:3]):  # Mostrar apenas os 3 primeiros
        print(f"\nDocumento {i+1}:")
        print(f"Fonte: {doc.metadata.get('source', 'N/A')}")
        print(f"Página: {doc.metadata.get('page', 'N/A')}")
        print(f"Conteúdo: {doc.page_content[:200]}...")
        
except Exception as e:
    print(f"Erro na consulta: {e}")

=== PERGUNTA ===

QUal é a jurisprudência atual sobre a LGPD ?


=== RESPOSTA ===
A jurisprudência sobre a Lei Geral de Proteção de Dados (LGPD) no Brasil ainda está em desenvolvimento, já que a lei é relativamente recente, tendo sido sancionada em 2018. No entanto, algumas tendências e decisões já começaram a se consolidar. Com base nos documentos fornecidos, posso destacar alguns pontos relevantes:

1. **Fundamentação da LGPD em Decisões Judiciais**: A LGPD tem sido invocada em casos relacionados à proteção de dados pessoais, complementando a legislação consumerista, como o Código de Defesa do Consumidor (CDC). Isso é exemplificado pelo uso da LGPD para argumentar sobre a segurança de dados pessoais e potenciais indenizações decorrentes de vazamentos de dados.

2. **Dados Sensíveis**: De acordo com o Art. 5º, II, da LGPD, dados considerados sensíveis exigem um tratamento diferenciado. A jurisprudência tem destacado a importância de classificar corretamente os dados para determinar o 

In [64]:
def consultar_lgpd(pergunta: str, chat_history: list = None):
    """
    Função para fazer consultas ao sistema RAG LGPD
    """
    if chat_history is None:
        chat_history = []
    
    try:
        result = retrieval_chain.invoke({
            "input": pergunta,
            "chat_history": chat_history
        })
        
        return {
            "pergunta": pergunta,
            "resposta": result["answer"],
            "documentos_fonte": len(result["context"]),
            "contexto": result["context"]
        }
    except Exception as e:
        return {
            "pergunta": pergunta,
            "resposta": f"Erro ao processar consulta: {e}",
            "documentos_fonte": 0,
            "contexto": []
        }

# Exemplo de uso da função
perguntas_teste = [
    "Quais são os direitos dos titulares de dados na LGPD?",
    "Qual é o prazo para responder ao titular sobre seus dados?",
    "Quais são as penalidades previstas na LGPD?",
    "O que diz a jurisprudência sobre consentimento na LGPD?"
]

print("=== TESTANDO MÚLTIPLAS CONSULTAS ===")
for pergunta in perguntas_teste[:2]:  # Testar apenas 2 para não sobrecarregar
    resultado = consultar_lgpd(pergunta)
    print(f"\n📋 PERGUNTA: {resultado['pergunta']}")
    print(f"📄 DOCUMENTOS: {resultado['documentos_fonte']}")
    print(f"✅ RESPOSTA: {resultado['resposta'][:300]}...")
    print("-" * 80)

=== TESTANDO MÚLTIPLAS CONSULTAS ===

📋 PERGUNTA: Quais são os direitos dos titulares de dados na LGPD?
📄 DOCUMENTOS: 13
✅ RESPOSTA: Os direitos dos titulares de dados pessoais estão previstos no art. 18 da Lei Geral de Proteção de Dados (LGPD). Esses direitos são fundamentais para garantir a proteção da privacidade e dos dados pessoais dos indivíduos. Veja os principais direitos listados na LGPD:

1. **Confirmação da existência ...
--------------------------------------------------------------------------------

📋 PERGUNTA: Qual é o prazo para responder ao titular sobre seus dados?
📄 DOCUMENTOS: 13
✅ RESPOSTA: A Lei Geral de Proteção de Dados Pessoais (LGPD) não estabelece de maneira explícita um prazo específico para que o controlador responda às solicitações dos titulares sobre seus dados. No entanto, o artigo 18 da LGPD assegura ao titular, dentre vários direitos, o de obter do controlador:

- Confirma...
-----------------------------------------------------------------------------